# Метрики ранжирования

In [1]:
from math import log2

from torch import Tensor, sort
import torch

## Число неверное расставленных пар (Kendall's Tau)

In [2]:
N = 20

ys_true = torch.rand(N)
ys_pred = torch.rand(N)

In [3]:
def num_swapped_pairs(ys_true: Tensor, ys_pred: Tensor) -> int:
    _, sorted_ys_true_idx = sort(ys_true, descending=True)

    sorted_preds_by_true = ys_pred[sort(ys_true, descending=True)[1]]
    count = 0
    
    for i in range(len(sorted_preds_by_true)):
        for j in range(i+1, len(sorted_preds_by_true)):
            if sorted_preds_by_true[i] < sorted_preds_by_true[j]:
                count += 1
    return count

In [4]:
discordant_number = num_swapped_pairs(ys_true, ys_pred)

In [5]:
total_pairs = N*(N-1)/2
kndltau = (total_pairs-2*discordant_number) / total_pairs

In [6]:
kndltau

-0.11578947368421053

In [7]:
from scipy import stats as sts
sts.kendalltau(ys_true, ys_pred).statistic

-0.11578947368421053

## DCG

In [8]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    assert gain_scheme in ['const', 'exp2']
    if gain_scheme == 'const':
        return y_value
    elif gain_scheme == 'exp2':
        return 2 * y_value - 1

In [9]:
def dcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str) -> float:
    dcg_value = 0
    _, sorted_ys_pred_idx = sort(ys_pred, descending=True)
    for i, rel in enumerate(ys_true[sorted_ys_pred_idx]):
        dcg_value += compute_gain(rel, gain_scheme=gain_scheme) / log2(i+2)
    return dcg_value

In [10]:
dcg(ys_true, ys_pred, gain_scheme='const')

tensor(2.8865)

In [11]:
from sklearn.metrics import dcg_score

In [12]:
dcg_score(ys_true.numpy().reshape(1, -1), ys_pred.numpy().reshape(1, -1))

2.88649124980886

## NDCG

In [13]:
def ndcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str = 'const') -> float:
    dcg_value = dcg(ys_true, ys_pred, gain_scheme=gain_scheme)
    perfect_dcg = dcg(ys_true, ys_true, gain_scheme=gain_scheme)
    
    return dcg_value / perfect_dcg

In [14]:
ndcg(ys_true, ys_pred)

tensor(0.7543)

In [15]:
from sklearn.metrics import ndcg_score

In [16]:
ndcg_score(ys_true.numpy().reshape(1, -1), ys_pred.numpy().reshape(1, -1))

0.754329570849857

## Precision at k

In [17]:
def precission_at_k(ys_true: Tensor, ys_pred: Tensor, k: int) -> float:
    _, sorted_ys_pred_idx = sort(ys_pred, descending=True)
    sorted_ys_pred_idx = sorted_ys_pred_idx[:k]

    top_k = ys_true[sorted_ys_pred_idx]
    if top_k.sum() == 0:
        return torch.tensor(-1)
    return top_k.mean()

In [18]:
precission_at_k((ys_true > 0.5).type(torch.float32), ys_pred, k=10)

tensor(0.6000)

In [19]:
(ys_true > 0.5).type(torch.float32)

tensor([0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
        1., 0.])

## MRR

In [20]:
def reciprocal_rank(ys_true: Tensor, ys_pred: Tensor) -> float:
    _, sorted_ys_pred_idx = sort(ys_pred, descending=True)
    sorted_true = ys_true[sorted_ys_pred_idx]

    return 1 / sort(sorted_true, descending=True)[1][0]

In [21]:
true_one = Tensor(N).type(torch.float32)
true_one.fill_(0)
true_one[4] = 1

In [22]:
true_one

tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])

In [23]:
reciprocal_rank(true_one, ys_pred)

tensor(1.)

## PFound

In [24]:
def p_found(ys_true: Tensor, ys_pred: Tensor, p_break: float = 0.15 ) -> float:
    _, sorted_ys_pred_idx = sort(ys_pred, descending=True)
    p_rel = ys_true[sorted_ys_pred_idx]
    # p_rel /= p_rel.max()

    p_look = Tensor(len(p_rel))
    p_look[0] = 1
    for i in range(1, len(p_rel)):
        p_look[i] = p_look[i-1] * (1 - p_rel[i-1]) * (1 - p_break)

    p_found_value = (p_look * p_rel).sum()

    return p_found_value

In [25]:
p_found(ys_true, ys_pred)

tensor(0.8131)

## Average Precision

In [26]:
def average_precision(ys_true: Tensor, ys_pred: Tensor) -> float:
    ap = 0
    _, sorted_ys_pred_idx = sort(ys_pred, descending=True)
    ys_true_ranked = ys_true[sorted_ys_pred_idx]

    recall = Tensor(len(ys_true_ranked))
    recall[0] = 0
    precision = Tensor(len(ys_true_ranked))
    for k in range(1, len(ys_true_ranked)):
        recall[k] = ys_true_ranked[:k].sum() / ys_true_ranked.sum()
        precision[k] = ys_true_ranked[:k].sum() / k
        ap += (recall[k] - recall[k-1]) * precision[k]

    if ap == 0:
        return torch.tensor(-1)

    return ap

In [27]:
average_precision((ys_true > 0.5).type(torch.float32), ys_pred)

tensor(0.5029)

In [28]:
from sklearn.metrics import average_precision_score

In [29]:
average_precision_score((ys_true > 0.5).type(torch.int32).numpy().reshape(1, -1), ys_pred.numpy().reshape(1, -1), average='micro')

0.5029048656499637